
# PROJET ML Garnier Grenier

Importation des données ELection 2022 2znd tour

In [6]:
# -*- coding: utf-8 -*-
# ESLint-disable
import pandas as pd
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import classification_report
from sklearn.feature_selection import VarianceThreshold

In [19]:
# Create a DataFrame from the CSV data
data = pd.read_excel('https://www.data.gouv.fr/fr/datasets/r/06d9816c-1b87-498d-985e-f312acee4f51')

df = data.copy()
df = df.drop(df.columns[[26, 27, 29]], axis=1)
df = df.drop(['Code du département', 'Code de la commune', 'N°Panneau', 'Sexe', 'Prénom'], axis=1,)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
df.head()

,Libellé du département,Libellé de la commune,Etat saisie,Inscrits,Abstentions,% Abs/Ins,Votants,% Vot/Ins,Blancs,% Blancs/Ins,% Blancs/Vot,Nuls,% Nuls/Ins,% Nuls/Vot,Exprimés,% Exp/Ins,% Exp/Vot,Nom,Voix,% Voix/Ins,% Voix/Exp,Unnamed: 28,Unnamed: 30,Unnamed: 31,Unnamed: 32
0,Ain,L'Abergement-Clémenciat,Complet,643,146,22.71,497,77.29,42,6.53,8.45,5,0.78,1.01,450,69.98,90.54,MACRON,237,36.86,52.67,LE PEN,213,33.13,47.33
1,Ain,L'Abergement-de-Varey,Complet,213,45,21.13,168,78.87,23,10.80,13.69,3,1.41,1.79,142,66.67,84.52,MACRON,94,44.13,66.20,LE PEN,48,22.54,33.80
2,Ain,Ambérieu-en-Bugey,Complet,8763,2448,27.94,6315,72.06,461,5.26,7.30,165,1.88,2.61,5689,64.92,90.09,MACRON,3080,35.15,54.14,LE PEN,2609,29.77,45.86
3,Ain,Ambérieux-en-Dombes,Complet,1282,266,20.75,1016,79.25,47,3.67,4.63,15,1.17,1.48,954,74.41,93.90,MACRON,451,35.18,47.27,LE PEN,503,39.24,52.73
4,Ain,Ambléon,Complet,103,19,18.45,84,81.55,12,11.65,14.29,0,0.00,0.00,72,69.90,85.71,MACRON,45,43.69,62.50,LE PEN,27,26.21,37.50


In [20]:
df.var(axis = 0)

/tmp/cache-anagarnier/ipykernel_202005/1637361517.py:1: FutureWarning: The default value of numeric_only in DataFrame.var is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.var(axis = 0)


Inscrits        8.762086e+07
Abstentions     7.612050e+06
% Abs/Ins       4.995080e+01
Votants         4.480993e+07
% Vot/Ins       4.995084e+01
Blancs          1.300221e+05
% Blancs/Ins    7.703465e+00
% Blancs/Vot    1.231380e+01
Nuls            1.326667e+04
% Nuls/Ins      7.354401e+00
% Nuls/Vot      1.221009e+01
Exprimés        3.883480e+07
% Exp/Ins       5.618798e+01
% Exp/Vot       2.461023e+01
Voix            2.476095e+07
% Voix/Ins      7.064450e+01
% Voix/Exp      1.314183e+02
Unnamed: 30     2.162080e+06
Unnamed: 31     8.296729e+01
Unnamed: 32     1.314279e+02
dtype: float64

/tmp/cache-anagarnier/ipykernel_202005/3365028497.py:1: FutureWarning: The default value of numeric_only in DataFrame.var is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  selector = df.var()


,Inscrits,Abstentions,% Abs/Ins,Votants,% Vot/Ins,Blancs,% Blancs/Ins,% Blancs/Vot,Nuls,% Nuls/Ins,% Nuls/Vot,Exprimés,% Exp/Ins,% Exp/Vot,Voix,% Voix/Ins,% Voix/Exp,Unnamed: 30,Unnamed: 31,Unnamed: 32
0,643,146,22.71,497,77.29,42,6.53,8.45,5,0.78,1.01,450,69.98,90.54,237,36.86,52.67,213,33.13,47.33
1,213,45,21.13,168,78.87,23,10.80,13.69,3,1.41,1.79,142,66.67,84.52,94,44.13,66.20,48,22.54,33.80
2,8763,2448,27.94,6315,72.06,461,5.26,7.30,165,1.88,2.61,5689,64.92,90.09,3080,35.15,54.14,2609,29.77,45.86
3,1282,266,20.75,1016,79.25,47,3.67,4.63,15,1.17,1.48,954,74.41,93.90,451,35.18,47.27,503,39.24,52.73
4,103,19,18.45,84,81.55,12,11.65,14.29,0,0.00,0.00,72,69.90,85.71,45,43.69,62.50,27,26.21,37.50


Proposition de visualisation

In [ ]:
# On souhaite prédire les resultat de l'elction 
# On est au second tour donc on a que 2 candidats MACRON et LE PEN Donc Deux classes
# On va donc utiliser un model de classification
# On va utiliser la regression logistique
